### Script to create Metadata Table

In [ ]:
%%sql
create table parallel.metadata(

    notebookid string,

    notebookname string,

    dependencies string,

    notebookparams string,

    pipeline string,

    type string,

    timeout string,

    retry string,

    retryIntervalInSeconds string,

    isactive boolean

) using delta

location "/data/common/parallel/metadata";

## Please copy all Necessary Pipeline Jsons from the Repo into Container

In [2]:
mssparkutils.fs.ls("abfss://eehrsisynapsefs@hrdatalakedev.dfs.core.windows.net/pipeline_jsons")

# Below Is the Command to Create Mount for the Container Where the Jsons are placed

In [15]:
 mssparkutils.fs.mount( 
     "abfss://eehrsisynapsefs@hrdatalakedev.dfs.core.windows.net", 
     "/pipelinejsons", 
     {"linkedService":"eehrsisynapsedev-WorkspaceDefaultStorage"} 
 ) 

## **Storing the JobId in a Variable**

In [16]:
a=mssparkutils.env.getJobId()

# Creating two Variables
   **1. Path Variable is to be used in mssparkutils****
   ****2. Folderpath variable to be used in os.listdir**

In [17]:
path = "synfs:/"+a+"/pipelinejsons/pipeline_jsons"
folderpath = "/synfs/"+a+"/pipelinejsons/pipeline_jsons"

In [18]:
mssparkutils.fs.ls(path) 

In [19]:
import json
import os
import csv

import pandas as pd

""" Using the Folderpath variable to Iterate all the Json files available in the Folder """

entries = os.listdir(folderpath)
inp_data = {}
reference_name = {}

for entry in entries:

    print(entry)

    data_JSON = folderpath+"/"+entry
    f = open(data_JSON)
    data_dict = json.load(f)

    inp_data['notebookid'] = []
    inp_data['notebookname'] = []
    inp_data['dependencies'] = []
    inp_data['notebookparams'] = []
    inp_data['pipeline'] = []
    inp_data['type'] = []
    inp_data['timeout'] = []
    inp_data['retry'] = []
    inp_data['retryIntervalInSeconds'] = []

    for data in data_dict['properties']['activities']:
        name = data['name']
        if data['type'] == "ExecutePipeline":
            reference_name.update({name: dict(dict(data).get('typeProperties'))['pipeline']['referenceName']})
        if data['type'] == "SynapseNotebook":
            reference_name.update({name: dict(dict(data).get('typeProperties'))['notebook']['referenceName']})

    for data in data_dict['properties']['activities']:
        params = {}
        # print(data)
        name = data['name']
        inp_data['notebookid'].append(name)
        if data['dependsOn']:
            inp_data['dependencies'].append(",".join([dependency['activity'] for dependency in data['dependsOn'] ]))
        else:
            inp_data['dependencies'].append('')
        if data['type'] == "ExecutePipeline":
            # reference_name.update({name: dict(dict(data).get('typeProperties'))['pipeline']['referenceName']})
            inp_data['notebookname'].append(dict(dict(data).get('typeProperties'))['pipeline']['referenceName'])
            inp_data['type'].append('pipeline')
            if 'parameters' in dict(dict(data).get('typeProperties')).keys():
                for key, value in dict(dict(data).get('typeProperties'))['parameters'].items():
                    params.update({key:value})
            inp_data['notebookparams'].append(params if params else '')
        if data['type'] == "SynapseNotebook":
            # reference_name.update({name:dict(dict(data).get('typeProperties'))['notebook']['referenceName']})
            inp_data['notebookname'].append(dict(dict(data).get('typeProperties'))['notebook']['referenceName'])
            inp_data['type'].append('notebook')
            if 'parameters' in dict(dict(data).get('typeProperties')).keys():
                for key, value in dict(dict(data).get('typeProperties'))['parameters'].items():
                    if type(value['value']) is dict:
                        params.update({key: value['value']['value']})
                    else:
                        params.update({key:value['value']})
            inp_data['notebookparams'].append(params if params else '')

        if 'policy' in data.keys():
            inp_data['timeout'].append(data['policy']['timeout'])
            inp_data['retry'].append(data['policy']['retry'])
            inp_data['retryIntervalInSeconds'].append(data['policy']['retryIntervalInSeconds'])
        else:
            inp_data['timeout'].append('')
            inp_data['retry'].append('')
            inp_data['retryIntervalInSeconds'].append('')
        inp_data['pipeline'].append(entry.split('.')[0])
        

    df = pd.DataFrame(inp_data)
    sp_df = spark.createDataFrame(df.astype(str))
    sp_df.createOrReplaceTempView("sp_df")
    spark.sql("""INSERT INTO parallel.metadata select * from sp_df""")


In [56]:
#%%sql
#CREATE table parallel.sample_metadata
#select * from sp_df;

In [5]:
# //   %%spark
# //   //  delete data in folders recursively equivalent as above drop table doesnot clean up data
# //   val fs = org.apache.hadoop.fs.FileSystem.get(sc.hadoopConfiguration)
# //   if(fs.exists(new org.apache.hadoop.fs.Path("abfss://eehrsisynapsefs@hrdatalakedev.dfs.core.windows.net/synapse/workspaces/eehrsisynapsedev/warehouse/parallel.db/sample_metadata"))) {
# //      mssparkutils.fs.rm("abfss://eehrsisynapsefs@hrdatalakedev.dfs.core.windows.net/synapse/workspaces/eehrsisynapsedev/warehouse/parallel.db/sample_metadata", true)
# // }

In [6]:
# //%%sql
#  TRUNCATE table parallel.sample_metadata

In [3]:
# -- %%sql
# -- drop table parallel.metadata;

In [5]:
%%sql
select * from parallel.metadata;

In [4]:
# -- %%sql
# -- create table parallel.metadata as 
# -- select * from parallel.sample_metadata ;